In this notebook we present the code used for the numerical simulations in Chapter 4.

In [1]:
import numpy as np
import scipy
import scipy.integrate as integrate 
import matplotlib.pyplot as plt
from pymor.basic import *

from pymor.core.logger import set_log_levels
set_log_levels({'pymor': 'ERROR'})

In [2]:
%matplotlib inline 

Hat functions:

In [3]:
def hat(x,i,I):
    h = I[1]-I[0]
    N = len(I)
    if i == 0:
        if x < I[i]:
            return 0
        elif x <= I[i+1]:
            return -(x-I[i+1])/h 
        else:
            return 0
    elif i <= N-2:
        if x < I[i-1]:
            return 0
        elif x <= I[i]:
            return (x-I[i-1])/h 
        elif x <= I[i+1]:
            return -(x-I[i+1])/h 
        else:
            return 0
    else:
        if x < I[i-1]:
            return 0 
        elif x <= I[i]:
            return (x-I[i-1])/h 
        else:
           return 0

We now simulate 
$\begin{align*}
-\Delta u(x) - L^{\alpha}u(x) &= \sin(x),\quad x\in (-2\pi,2\pi),\\
u(-2\pi) = u(2\pi) &= 0.
\end{align*}$
This problem is well posed on $H^1_0(\Omega)$ for each $\alpha \in(0,1)$. We take the $H^1$ seminorm which is a norm on $H^1_0$ equivalent to the usual Sobolev norm. Using this norm the inner product matrix is equal to the stiffnes matrix associated to the bilinearform.\
We begin by setting up the stiffnes matrix for the Laplace operator:
$\begin{align*}
a(\phi_i,\phi_j)=\int_a^b \phi_i'\phi_j' dx
\end{align*}$
by definition this is equal to the $H^1_0$ seminorm, i.e. we can use both for pyMOR.

We now define the stiffnes matrix for the laplacian with homogenous boundary conditions ($u=0$ on $\partial\Omega$):

In [4]:
def H1_semiprod(I):
    N = len(I)
    h = I[1]-I[0]
    diag1 = np.ones(N-2)
    diag2 = np.ones(N-3)
    A = np.diagflat(2*diag1,0) + np.diagflat(-diag2,1) +np.diagflat(-diag2,-1)
    return (1/h)*A

Now determine the stiffnes matrix for $L^{\alpha}$. In view of the readability of the code we define some auxilliary functions.

In [5]:
def innerint_1(y,i,I,alpha):
    return integrate.quad(lambda x: hat(x,i,I)*(y-x)**(-alpha),I[i],y)[0]+integrate.quad(lambda x: hat(x,i,I)*(x-y)**(-alpha),y,I[i+1])[0]

def innerint_2(y,i,I,alpha):
    val = integrate.quad(lambda x: hat(x,i+1,I)*(y-x)**(-alpha),I[i],y)[0]+integrate.quad(lambda x: hat(x,i+1,I)*(x-y)**(-alpha),y,I[i+1])[0]
    return val

def seconddiag(i,I,alpha):
    return integrate.quad(lambda y: hat(y,i,I)*innerint_2(y,i,I,alpha),I[i],I[i+1])[0]\
                        + integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,i+1,I)*(x-y)**(-alpha),I[i],I[i+1],I[i+1],I[i+2])[0]\
                            +integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,i+1,I)*(x-y)**(-alpha),I[i-1],I[i],I[i],I[i+2])[0]
                    
def firstdiag(i,I,alpha):
    i2 = integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,i,I)*(x-y)**(-alpha),I[i-1],I[i],I[i],I[i+1])[0]
    i3 = integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,i,I)*(y-x)**(-alpha),I[i],I[i+1],I[i-1],I[i])[0]
    i4 = integrate.quad(lambda y: hat(y,i,I)*innerint_1(y,i,I,alpha),I[i],I[i+1])[0]
    i1 = integrate.quad(lambda y: hat(y,i,I)*innerint_2(y,i-1,I,alpha),I[i-1],I[i])[0]
    return i1+i2+i3+i4

In [6]:
def stiff1(I,alpha):   

    N = len(I)
    B = np.zeros((N,N))
    val0 = firstdiag(1,I,alpha)
    val1 = seconddiag(1,I,alpha)
    
    for i in range(N-1):
            if i == 0:
                B[i,i] = (1/(1-alpha))*integrate.quad(lambda y: hat(y,i,I)**2*((y-I[0])**(1-alpha)+(I[-1]-y)**(1-alpha)),I[0],I[1])[0]\
                    - integrate.quad(lambda y: hat(y,0,I)*innerint_1(y,0,I,alpha),I[0],I[1])[0]
                B[i,i+1] = (1/(1-alpha))*integrate.quad(lambda y: hat(y,i,I)*hat(y,i+1,I)*((y-I[0])**(1-alpha)+(I[-1]-y)**(1-alpha)),I[0],I[1])[0]\
                    -(integrate.quad(lambda y: hat(y,0,I)*innerint_2(y,0,I,alpha),I[0],I[1])[0]\
                        + integrate.dblquad(lambda x,y: hat(y,0,I)*hat(x,1,I)*(x-y)**(-alpha),I[0],I[1],I[1],I[2])[0])
                for j in range(2,N-1):
                    B[i,j] = -integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,j,I)*(x-y)**(-alpha),I[0],I[1],I[j-1],I[j+1])[0]
                B[i,N-1] = -integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,N-1,I)*(x-y)**(-alpha),I[0],I[1],I[N-2],I[N-1])[0]
            elif i == 1:
                B[i,i] = (1/(1-alpha))*integrate.quad(lambda y: hat(y,i,I)**2*((y-I[0])**(1-alpha)+(I[-1]-y)**(1-alpha)),I[i-1],I[i+1])[0]\
                        -val0
                B[i,i+1] = (1/(1-alpha))*integrate.quad(lambda y: hat(y,i,I)*hat(y,i+1,I)*((y-I[0])**(1-alpha)+(I[-1]-y)**(1-alpha)),I[i],I[i+1])[0]\
                       - val1
                for j in range(3,N-1):
                    B[i,j] = -integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,j,I)*(x-y)**(-alpha),I[i-1],I[i+1],I[j-1],I[j+1])[0]
                B[i,N-1] = -integrate.dblquad(lambda x,y: hat(y,i,I)*hat(x,N-1,I)*(x-y)**(-alpha),I[0],I[2],I[N-2],I[N-1])[0]
            else:
                B[i,i] = (1/(1-alpha))*integrate.quad(lambda y: hat(y,i,I)**2*((y-I[0])**(1-alpha)+(I[-1]-y)**(1-alpha)),I[i-1],I[i+1])[0]\
                    -val0
                B[i,i+1] = (1/(1-alpha))*integrate.quad(lambda y: hat(y,i,I)*hat(y,i+1,I)*((y-I[0])**(1-alpha)+(I[-1]-y)**(1-alpha)),I[i],I[i+1])[0]\
                    -val1

            B[N-2,N-1] = B[0,1]       
            B[N-1,N-1] = B[0,0]

    for i in range(N):
            if i == 0 or i == 1:
                continue
            elif i == N-2:
                continue
            else:
                B[i,i+2:] = B[i-1,i+1:N-1]

    B = B + B.T - np.diag(np.diagonal(B))
    return(B)

Set up the right hand-side vector

In [5]:
def set_rhs(I): 

    N = len(I)
    rhs = np.zeros((1,N-2)).T

    for i in range(1,N-1):
        if i == 0:
            rhs[i] = 0
        elif i == N-1:
            rhs[i] = 0
        else:
            rhs[i-1] = integrate.quad(lambda x: hat(x,i,I)*(np.sin(x)),I[i-1],I[i+1])[0]
    return rhs

Now we investigate the convergence rate of the simulated solution against the analytic solution, plotted in the $H^1$ semi-norm.

In [6]:
#calculation the squared H^1_0 error for sinus
def H1_0err(u_h,I):
    h = I[1]-I[0]
    err = 0
    for k in range(len(I)-1):
        tmp = (u_h[k+1] - u_h[k])/h
        err += (1/4)*(-2*I[k]+2*I[k+1]-np.sin(2*I[k])+np.sin(2*I[k+1])) - 2*tmp*(np.sin(I[k+1])-np.sin(I[k])) + tmp**2*h
    print(err)
    return err


In [7]:
N_range = np.array([8,16,32,64,128,256,512,1024,2048,4096,8192,16384,32768])
k = 0
err_H1 = np.zeros(N_range.shape)
for N in N_range:
    N = int(N)
    I_N = np.linspace(-2*np.pi,2*np.pi,N)
    u_h = np.zeros((N,1))
    A = H1_semiprod(I_N)
    rhs = set_rhs(I_N)
    u_h[1:N-1] = scipy.linalg.solve(A,rhs,sym_pos=True)
    err_H1[k] = H1_0err(u_h,I_N)
    k += 1

plt.figure(0)
plt.loglog(N_range,err_H1,'-x',label ='$H^1_0$ error')
plt.grid(True)
plt.xlabel('$\mathcal{N}$')
plt.ylabel('$||u-u_h||_{H^1_0}^2$')
plt.legend()

We start investigating the reducibility of the problem. In the thesis, we first investigated the reducibility for $\alpha = 0.2$. Then we investigated if there is a connection between the reducibility, and the order of singularity.\
 The code for those experiments is the same, one only has to define for which $\alpha$'s the model should be considered, stored in the vector 'alpha'.

In [8]:
I = np.linspace(-2*np.pi,2*np.pi,2000)
A1 = H1_semiprod(I)
N = len(I)

#define the singularity order for which the model should be considered.
alpha = [0.2,0.4,0.6,0.7,0.85]

A2 = []
for a in alpha:
    tmp = stiff1(I,a)
    A2.append(tmp[1:N-1,1:N-1])


Set up the problem in pyMOR

In [9]:
H10_prod = NumpyMatrixOperator(H1_semiprod(I))

rhs = set_rhs(I)
rhs_op = NumpyMatrixOperator(rhs)

coefs = [ProjectionParameterFunctional('mu', 2,0),
            ProjectionParameterFunctional('mu',2,1)]

fom = []
for i,a in enumerate(alpha):
    ops = [NumpyMatrixOperator(A1),NumpyMatrixOperator(A2[i])]
    op = LincombOperator(ops,coefs)
    fom.append(StationaryModel(op,rhs_op))

Plot the approximated solution for some parameters

In [10]:
fig0 = plt.figure(0)
for i,a in enumerate(alpha):
    U = fom[i].solve(Mu({'mu':[0.5,0.5]}))
    u = U.to_numpy()[0]
    plt.plot(I[1:N-1],u,label = r'$\alpha =$ %.2f' %(a))
plt.grid(True)
plt.legend()

Set up a coerive reductor

In [11]:
reductor = []
for i,a in enumerate(alpha):
    reductor.append(CoerciveRBReductor(
                        fom[i],
                        product = H10_prod,
                ))

Generate the reduced basis

In [12]:
n_train = 300
atol = 1e-10
N_max = 50
parameter_space = []
greedy_data = []


low_bounds = [0.001,1]
upp_bounds = [0.001,1]
par_bounds = {'mu':low_bounds,'mu':upp_bounds}
train_sample= fom[0].parameters.space(par_bounds).sample_randomly(n_train)

for i,a in enumerate(alpha):
    parameter_space.append(fom[i].parameters.space(0.001,1))
    greedy_data.append(rb_greedy(fom[i], reductor[i], train_sample,
                                max_extensions=N_max, atol=atol))


Plot the training error

In [13]:
train_err = []
for i,a in enumerate(alpha):
    train_err.append(greedy_data[i]['max_errs'])

for i,a in enumerate(alpha):
    plt.semilogy(range(greedy_data[i]['extensions']+1),train_err[i], label = r'$\alpha =$ %.2f' %(a))
plt.legend()
plt.xlabel('$N$')
plt.ylabel('$max_{\mu} ||u_h(\mu)-u_N(\mu)||$')
plt.grid(which='both')
plt.title('Training error in  $|\cdot|_{H^1_0}$')

Build the reduced models

In [14]:
rom = []
bases = []
for i,a in enumerate(alpha):
    rom.append(reductor[i].reduce())
    bases.append(reductor[i].bases['RB'])

Set up the test set

In [15]:
n_test = 300
test_set = []
for i,a in enumerate(alpha):
    test_set.append(parameter_space[i].sample_randomly(n_test))
    
test_sample = fom[0].parameters.space(par_bounds).sample_randomly(n_test)

Compute the test absolute and relativ test error estimation as well as the exact error

In [16]:
test_err_est = []
test_err_est_rel = []
test_err = []


for i,a in enumerate(alpha):
        test_err_est.append(np.zeros(len(bases[i])))
        test_err.append(np.zeros(len(bases[i])))
        test_err_est_rel.append(np.zeros(len(bases[i])))

        #determine exact sol for each alpha                                     #for the exact train error 
        sol_ex = []
        for j in range(len(test_set[i])):
                # sol_ex.append(fom[i].solve(test_set[i][j]))
                sol_ex.append(fom[i].solve(test_sample[j]))

        #set up reduced model for each dim = 1,...,N
        for dim in range(len(test_err_est[i])):
                reductor_tmp = CoerciveRBReductor(reductor[i].fom,RB = bases[i][:dim+1], product = reductor[i].products['RB'], 
                coercivity_estimator = reductor[i].coercivity_estimator,check_orthonormality=reductor[i].check_orthonormality,
                        check_tol=reductor[i].check_tol)
                rom_tmp = reductor_tmp.reduce()
                
                #compute err estimate and ex err absolute and relative
                test_err_est_tmp = np.zeros(n_test)
                test_err_est_rel_tmp = np.zeros(n_test)
                test_err_tmp = np.zeros(n_test)
                train_err_rel_tmp = np.zeros(n_test)

                for j in range(len(test_set[i])):
                        # test_err_est_tmp[j] = rom_tmp.estimate_error(test_set[i][j])          #determine train err_estimate
                        test_err_est_tmp[j] = rom_tmp.estimate_error(test_sample[j])
                        test_err_est_rel_tmp[j] = test_err_est_tmp[j]/np.square(reductor_tmp.products['RB'].apply2(sol_ex[j],sol_ex[j]))
                        
                        # sol_rb = rom_tmp.solve(test_set[i][j])
                        sol_rb = rom_tmp.solve(test_sample[j])
                        diff = reductor_tmp.reconstruct(sol_rb) - sol_ex[j]
                        test_err_tmp[j] = np.square(reductor_tmp.products['RB'].apply2(diff,diff))      #determine train err_ex       

                test_err_est[i][dim] = np.max(test_err_est_tmp)
                test_err_est_rel[i][dim] = np.max(test_err_est_rel_tmp)
                test_err[i][dim] = np.max(test_err_tmp)



plot the figures, stored in 'alpha02' and 'alphas'

In [17]:
fig0 = plt.figure(0)
for i,a in enumerate(alpha):
    plt.semilogy(range(len(bases[i])),test_err[i],label = 'truth test error')# 'r'$\alpha = $%.2f' %(alpha[i])
    #p =plt.semilogy(range(greedy_data[i]['extensions']+1),train_err[i],'-', label = 'test error est')#label = r'$\alpha = $%.2f' %(alpha[i]))
    plt.semilogy(range(len(bases[i])),test_err_est[i],label = 'test err est') #,color = p[0].get_color())
plt.legend()
plt.xlabel('$N$')
plt.grid(True)


fig1 = plt.figure(1)
for i,a in enumerate(alpha):
    p =plt.semilogy(range(greedy_data[i]['extensions']+1),train_err[i], label = 'train error')#, label = r'$\alpha = $%.2f' %(alpha[i]))
    plt.semilogy(range(len(bases[i])),test_err_est[i],label = 'test error') #,color = p[0].get_color())
plt.grid(True)
plt.legend()
plt.xlabel('$N$')


fig2 = plt.figure(2)
for i,a in enumerate(alpha):
    plt.semilogy(range(len(bases[i])),test_err_est_rel[i], label = 'rel test error')
plt.grid(True)
plt.legend()
plt.xlabel('$N$')

plt.figure(3)
for i,a in enumerate(alpha):

    plt.semilogy(range(len(bases[i])),test_err_est_rel[i], label = 'rel test error')
    plt.semilogy(range(len(bases[i])),test_err_est[i],label = 'test error')
plt.grid(True)
plt.legend()